In [113]:
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from matplotlib import rcParams
from seaborn import heatmap
import seaborn as sns
import xarray as xr
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import cartopy.feature as cfeature
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import NaturalEarthFeature, COLORS
from matplotlib.offsetbox import AnchoredText
import cartopy.crs as ccrs
from scipy import stats

In [114]:
"""Plotting parameters"""
#rcParams['axes.formatter.limits'] = (-7, 7)
rcParams['axes.formatter.limits'] = (-2,3)
rcParams['axes.labelsize'] = 16
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['font.size'] = 16
rcParams['figure.titlesize'] = 16
rcParams['axes.titlesize'] = 16
rcParams['legend.fontsize'] = 16
%matplotlib inline

### Master spreadsheet from Mike Novak 

In [183]:
df = pd.read_excel('/storage/data3/disk04/CarbonFluxes/Data/Master_Cruise_File_Novak_9Aug2017.xlsx')

In [184]:
df.describe()

,Scan #,Flow_Temp1,Flow_Temp2,Flow_Sal,Fdom(flowthrough),Trans(flowthrough),beam c(flowthrough),lat(flowthrough),lon(flowthrough),Flow_chl(flowthrough),'STATION','DATE','DEPTH'
count,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1.326000e+03,1326.000000
mean,5354.941590,-717.765163,-774.245105,-712.918794,-720.336966,-718.110233,-743.493097,-710.462120,-741.299232,-799.811949,-623.969306,6.852899e+06,-646.250603
std,25021.341942,453.938434,418.578583,461.755830,449.782646,476.667950,436.139344,465.721732,415.950307,399.657409,513.141550,9.534283e+06,481.840398
min,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02,-999.000000
25%,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02,-999.000000
50%,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02,-999.000000
75%,-999.000000,10.221450,-999.000000,30.213635,3.577573,60.314935,0.077460,37.394953,-75.054542,-999.000000,37.000000,2.010045e+07,2.800000
max,165849.000000,25.776850,25.134940,36.171140,10.812200,98.749580,5.210900,44.299160,-65.725040,6.335000,220.000000,2.012082e+07,90.500000


In [185]:
df = df.iloc[1:] # drop 2nd row with units
df.head()

,sample_id,year,Month,day,Time SB,hr,min,sec,cruise,station,...,'CYANOBACTERIA','MICROFLAGELLATE_LT10UM','MICROFLAGELLATE_GT10UM','CHOANOFLAGELLATE','CILIATE_ALORICATE','CILIATE_MESODINIUM','CILIATE_TINTINNID','TOTAL_PHYTOPLANKTON','TOTAL_MICROFLAGELLATES','TOTAL_MICROZOOPLANKTON'
1,100,2009,8,17,0.938889,22,32,0,CV1,1,...,-999,98560,-999,-999,-999,-999,-999,83550,98560,-999
2,101,2009,8,17,0.938889,22,32,0,CV1,1,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,102,2009,8,17,0.938889,22,32,0,CV1,1,...,-999,119000,-999,-999,-999,-999,-999,119590,119000,-999
4,103,2009,8,17,0.938889,22,32,0,CV1,1,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
5,104,2009,8,18,0.53125,12,45,0,CV1,6,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [186]:
# get selected variables (note Rrs values not available)
# sample_id, year, Month, day, hr, min, lat, lon, water_depth, depth_corrected_for_Niskin_offset, 
# Temp_CTD, Sal_CTD, ag412, S275:295, DOC, POC

In [187]:
# select data columns of interest
df = df[['sample_id', 'year', 'Month', 'day', 'hr', 'min', 'lat', 'lon', 'water_depth', 
    'depth_corrected_for_Niskin_offset', 'Temp_CTD', 'Sal_CTD', 'ag412', 'S275:295', 'DOC', 'POC_Exp',
        'POC_Cetenic_(19.1ug)']]
df.rename(columns={'depth_corrected_for_Niskin_offset': 'depth', 'water_depth': 'bdepth', 'Month': 'month', 
                  'POC_Exp': 'POC1', 'POC_Cetenic_(19.1ug)': 'POC2', 'Temp_CTD': 'Wt', 
                  'Sal_CTD': 'Sal'}, inplace=True)
df.head()

,sample_id,year,month,day,hr,min,lat,lon,bdepth,depth,Wt,Sal,ag412,S275:295,DOC,POC1,POC2
1,100,2009,8,17,22,32,41.314,-70.9444,34,1.43,21.0591,31.3975,0.17225,0.0250035,100.982,221.223,231.682
2,101,2009,8,17,22,32,41.314,-70.9444,34,4.43,20.9185,31.396,0.184552,0.0241851,144.43,209.834,224.873
3,102,2009,8,17,22,32,41.314,-70.9444,34,13.53,18.9376,31.4249,0.200528,0.0234506,122.68,179.686,193.121
4,103,2009,8,17,22,32,41.314,-70.9444,34,28.63,13.9983,32.1005,0.182454,0.0236282,107.59,174.993,188.428
5,104,2009,8,18,12,45,40.304,-71.8109,73,1.23,22.7161,0.49803,0.0745892,0.0318233,80.1981,128.53,138.988


In [188]:
# convert POC in mg/m3 to mmol/m3 to match units of DOC
df.loc[:,'POC1'] /= 12
df.loc[:,'POC2'] /= 12

In [189]:
df.head()

,sample_id,year,month,day,hr,min,lat,lon,bdepth,depth,Wt,Sal,ag412,S275:295,DOC,POC1,POC2
1,100,2009,8,17,22,32,41.314,-70.9444,34,1.43,21.0591,31.3975,0.17225,0.0250035,100.982,18.4353,19.3068
2,101,2009,8,17,22,32,41.314,-70.9444,34,4.43,20.9185,31.396,0.184552,0.0241851,144.43,17.4862,18.7394
3,102,2009,8,17,22,32,41.314,-70.9444,34,13.53,18.9376,31.4249,0.200528,0.0234506,122.68,14.9739,16.0934
4,103,2009,8,17,22,32,41.314,-70.9444,34,28.63,13.9983,32.1005,0.182454,0.0236282,107.59,14.5828,15.7023
5,104,2009,8,18,12,45,40.304,-71.8109,73,1.23,22.7161,0.49803,0.0745892,0.0318233,80.1981,10.7108,11.5823


In [190]:
df.describe()

,sample_id,year,month,day,hr,min,lat,lon,bdepth,depth,Wt,Sal,ag412,S275:295,DOC,POC1,POC2
count,1325,1325,1325,1325,1325,1325,1325.000000,1325.000000,1325.0,1325.0,1325.0,1325.0,1325.0,1325.0,1325.0,1321.00,1321.00
unique,1325,15,6,31,24,60,587.000000,586.000000,385.0,473.0,754.0,818.0,1199.0,1200.0,1217.0,1201.00,1200.00
top,1424,2010,6,16,17,5,38.732148,-76.458868,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-83.25,-83.25
freq,1,372,330,80,141,41,15.000000,15.000000,100.0,241.0,509.0,328.0,118.0,117.0,67.0,112.00,112.00


In [199]:
df[df<-900] = np.nan

In [200]:
df.describe()

,sample_id,year,month,day,hr,min,lat,lon,bdepth,depth,Wt,Sal,ag412,S275:295,DOC,POC1,POC2
count,1325,1325,1325,1325,1325,1325,1325.000000,1325.000000,1224.00,1135.0,816.0,997.0,1207.0,1208.0,1258.0,1321.00,1321.00
unique,1325,15,6,31,24,60,587.000000,586.000000,383.00,472.0,753.0,817.0,1198.0,1199.0,1216.0,1201.00,1200.00
top,1424,2010,6,16,17,5,38.732148,-76.458868,5.18,0.0,10.4,11.2,0.0,0.0,208.1,-83.25,-83.25
freq,1,372,330,80,141,41,15.000000,15.000000,21.00,241.0,6.0,11.0,10.0,10.0,3.0,112.00,112.00


In [212]:
df.to_pickle('novak_v1.pkl')

### Dirk Aurin's database

In [195]:
# Importing the dataset
# Data from Dirk Aurin's matchup file
# see /accounts/sergio/Algorithm Development/MatchupsDirkAurin.m
fp = '/accounts/sergio/Algorithm Development/aurin_matchup_data.mat'
mat = loadmat(fp) # load mat-file

In [196]:
#cols = ['lat', 'lon', 'depth', 'Rrs412', 'Rrs443', 'Rrs488', 'Rrs531', 'Rrs547', 'Rrs667',
#        'ag412', 'S275_295', 'Wt', 'Sal', 'DOC', 'POC']
#cols = ['sample_id', 'datetime', 'year', 'month', 'day', 'hr', 'min', lat', 'lon', 'bdepth', 'depth', 'Rrs443', 'Rrs547', 'Rrs667', 'Wt', 'Sal', 'ag412', 'S275_295', 'DOC', 'POC']
# no Rrs to match Novak's data base
cols = ['sample_id', 'year', 'month', 'day', 'hr', 'min', 'lat', 'lon', 'bdepth', 'depth', 'Wt', 'Sal', 'ag412', 'S275_295', 'DOC', 'POC']

In [197]:
mat_data = {}
for col in cols:
    mat_data[col] = mat[col].reshape(-1,)
idx = np.arange(mat_data[col].shape[0])

In [198]:
df2 = pd.DataFrame(mat_data, index=idx)
df2 = df2[cols]
df2.rename(columns={'S275_295': 'S275:295'}, inplace=True)
df2.describe()

,sample_id,year,month,day,hr,min,lat,lon,bdepth,depth,Wt,Sal,ag412,S275:295,DOC,POC
count,51192.000000,51192.000000,51192.000000,51192.000000,51192.000000,51192.000000,51192.000000,51192.000000,51086.000000,51188.000000,47432.000000,47542.000000,48378.000000,2480.000000,1957.000000,625.000000
mean,26634.059482,2006.181786,8.107595,15.566260,13.585267,29.270824,25.762056,-53.240540,2324.549258,3.927860,17.821336,34.079622,0.120203,0.026912,88.845591,49.177807
std,17918.447542,3.452806,2.663032,8.927259,6.435201,17.386018,27.725959,35.455600,2284.082682,1.329138,7.599141,2.636865,0.133336,0.007396,38.723399,72.635677
min,1.000000,1984.000000,1.000000,1.000000,0.000000,0.000000,-70.999500,-177.990500,0.083333,0.000000,-1.863100,0.000000,0.004966,0.013937,42.584760,0.125000
25%,12798.750000,2003.000000,6.000000,7.000000,9.000000,14.000000,8.515577,-68.544732,124.805557,3.000000,12.556400,32.100000,0.049883,0.021710,68.827032,11.836570
50%,25596.500000,2006.000000,9.000000,16.000000,15.000000,29.000000,40.450067,-66.289759,1305.527832,5.000000,17.090000,34.658300,0.096300,0.024787,76.336910,19.772009
75%,38394.250000,2009.000000,10.000000,23.000000,19.000000,44.000000,43.711893,-27.615472,4652.374878,5.000000,25.155250,36.255275,0.145000,0.030366,93.749009,39.766667
max,101065.000000,2012.000000,12.000000,31.000000,23.000000,59.000000,78.770670,179.545384,7498.000000,10.000000,30.986089,37.740700,2.457448,0.045639,272.500000,370.405833


In [211]:
df2.to_pickle('./aurin_v1.pkl')

### Merge data sets

In [203]:
dfa = df[['year','month','day','hr','min','lat','lon', 'bdepth','depth','Wt','Sal','ag412','S275:295','DOC']]
dfb = df2[['year','month','day','hr','min','lat','lon', 'bdepth','depth','Wt','Sal','ag412','S275:295','DOC']]

In [207]:
dfmerged = dfa.append([dfb])

In [208]:
dfmerged

,year,month,day,hr,min,lat,lon,bdepth,depth,Wt,Sal,ag412,S275:295,DOC
1,2009,8,17,22,32,41.314,-70.9444,34,1.43,21.0591,31.3975,0.17225,0.0250035,100.982
2,2009,8,17,22,32,41.314,-70.9444,34,4.43,20.9185,31.396,0.184552,0.0241851,144.43
3,2009,8,17,22,32,41.314,-70.9444,34,13.53,18.9376,31.4249,0.200528,0.0234506,122.68
4,2009,8,17,22,32,41.314,-70.9444,34,28.63,13.9983,32.1005,0.182454,0.0236282,107.59
5,2009,8,18,12,45,40.304,-71.8109,73,1.23,22.7161,0.49803,0.0745892,0.0318233,80.1981
6,2009,8,18,12,45,40.304,-71.8109,73,4.13,22.5705,11.2086,0.073434,0.0315459,107.759
7,2009,8,18,12,45,40.304,-71.8109,73,30.13,19.7717,32.9197,0.141971,0.0251785,86.9364
8,2009,8,18,12,45,40.304,-71.8109,73,69.13,11.643,34.3878,0.133717,0.0246366,77.8488
9,2009,8,18,15,30,40.0969,-71.7587,82,1.23,23.8189,32.2517,0.0764739,0.0317688,80.4922
10,2009,8,18,15,30,40.0969,-71.7587,82,4.53,23.689,32.2813,0.0876821,0.0301856,89.2125


In [209]:
dfmerged.to_pickle('./novak_aurin_merged_v1.pkl')